In [3]:
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pyhwp

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -U langchain-chroma

In [8]:
import os
import subprocess
from bs4 import BeautifulSoup
import re
from typing import List, Dict, Any
import json
import tiktoken

########################################
# 1. HWP → XHTML 변환 함수
########################################
def convert_all_hwps_in_folder(input_directory, output_directory, hwp5proc_executable="/home/spai0323/myenv/bin/hwp5proc"):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    converted_files = []
    for filename in os.listdir(input_directory):
        if filename.lower().endswith(".hwp"):
            input_path = os.path.join(input_directory, filename)
            base_name, _ = os.path.splitext(filename)
            output_path = os.path.join(output_directory, f'{base_name}.xhtml')

            command = [hwp5proc_executable, "xml", input_path]

            try:
                result = subprocess.run(command, capture_output=True, text=True, check=True)
                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(result.stdout)
                print(f"✅ 변환 성공: {filename}")
                converted_files.append(output_path)
            except Exception as e:
                print(f"❌ 변환 실패 {filename}: {e}")
    return converted_files

########################################
# 2. XHTML 파싱 함수
########################################
def parse_hwp5proc_xhtml(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()
    soup = BeautifulSoup(content, 'xml')

    parsed_data = {'sections': [], 'tables': [], 'metadata': {}}

    # 표 추출
    tables = soup.find_all('TableControl')
    for i, table in enumerate(tables):
        table_data = extract_table_structure(table, table_id=i)
        if table_data:
            parsed_data['tables'].append(table_data)

    # 문단 추출
    paragraphs = soup.find_all('Paragraph')
    for i, para in enumerate(paragraphs):
        text_content = extract_paragraph_text(para, para_id=i)
        if text_content and text_content.get('content', '').strip():
            parsed_data['sections'].append(text_content)

    parsed_data['metadata'] = {
        'total_paragraphs': len(parsed_data['sections']),
        'total_tables': len(parsed_data['tables']),
        'file_size': len(content)
    }
    return parsed_data

def extract_table_structure(table_element, table_id):
    table_data = {'id': table_id, 'rows': [], 'raw_content': ''}
    table_body = table_element.find('TableBody')
    if not table_body:
        return None
    for row in table_body.find_all('TableRow'):
        row_data = []
        for cell in row.find_all('TableCell'):
            text = " ".join([t.string.strip() for t in cell.find_all('Text') if t.string])
            row_data.append(text)
        if any(row_data):
            table_data['rows'].append(row_data)
    table_data['raw_content'] = convert_table_to_natural_language(table_data['rows'])
    return table_data

def extract_paragraph_text(paragraph_element, para_id):
    text_content = " ".join([t.string.strip() for t in paragraph_element.find_all('Text') if t.string])
    return {'id': para_id, 'type': 'paragraph', 'content': text_content}

def convert_table_to_natural_language(table_rows):
    if not table_rows or len(table_rows) < 2:
        return ""
    headers, *rows = table_rows
    natural_texts = []
    for row in rows:
        row_text = [f"{h}: {c}" for h, c in zip(headers, row) if c.strip()]
        if row_text:
            natural_texts.append(", ".join(row_text))
    return ". ".join(natural_texts)

########################################
# 3. 청킹 함수
########################################
def create_structured_chunks(parsed_data: Dict[str, Any], max_chunk_size: int = 100) -> List[Dict[str, Any]]:
    chunks = []
    for table in parsed_data['tables']:
        if table['raw_content']:
            chunks.append({'type': 'table', 'content': table['raw_content']})
    for section in parsed_data['sections']:
        content = section['content']
        if len(content) <= max_chunk_size:
            chunks.append({'type': 'paragraph', 'content': content})
        else:
            chunks.extend(split_large_paragraph_by_tokens(content, max_chunk_size))
    for i, chunk in enumerate(chunks):
        chunk['chunk_id'] = f"chunk_{i:04d}"
    return chunks

def split_large_paragraph_by_tokens(content: str, max_chunk_size: int):
    encoding = tiktoken.get_encoding("cl100k_base") # embedding-3-small 모델에 사용되는 인코딩
    sentences = re.split(r'[.!?]\s+', content)
    chunks, current_tokens, current_chunk = [], 0, []

    for s in sentences:
        sentence_tokens = len(encoding.encode(s))
        if current_tokens + sentence_tokens <= max_chunk_size:
            current_chunk.append(s)
            current_tokens += sentence_tokens
        else:
            chunks.append({'type': 'paragraph', 'content': ". ".join(current_chunk)})
            current_chunk, current_tokens = [s], sentence_tokens

    if current_chunk:
        chunks.append({'type': 'paragraph', 'content': ". ".join(current_chunk)})
    return chunks

########################################
# 4. json 파일 생성 및 저장
########################################
import json
import os

def save_chunks_to_json(chunks, source_file, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    base_name = os.path.splitext(os.path.basename(source_file))[0]
    output_path = os.path.join(output_dir, f"{base_name}.json")

    # 청크에 메타데이터 추가
    for chunk in chunks:
        chunk["source_file"] = base_name

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)

    print(f"💾 저장 완료: {output_path}")

########################################
# 5. 전체 실행 메인
########################################
def main_pipeline():
    input_folder = "../data/raw/files"
    output_folder = "../data/processed/bydatapreprocessingbjs(0922)/all_xhtml"
    os.makedirs(output_folder, exist_ok=True)
    json_output_dir = "../data/processed/bydatapreprocessingbjs(0922)/json"
    
    # 1. 변환
    converted_files = convert_all_hwps_in_folder(input_folder, output_folder)

    # 2. 파싱 + 3. 청킹
    for file_path in converted_files:
        print(f"\n=== 처리 중: {file_path} ===")
        parsed_data = parse_hwp5proc_xhtml(file_path)
        chunks = create_structured_chunks(parsed_data)
        print(f"문단 {len(parsed_data['sections'])}, 표 {len(parsed_data['tables'])}, 청크 {len(chunks)}개 생성")

        save_chunks_to_json(chunks, file_path, json_output_dir)

if __name__ == "__main__":
    main_pipeline()

✅ 변환 성공: 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구.hwp
✅ 변환 성공: 케빈랩 주식회사_평택시 강소형 스마트시티 AI 기반의 영상감시 시스템 .hwp
✅ 변환 성공: 수협중앙회_수협중앙회 수산물사이버직매장 시스템 재구축 ISMP 수립 입.hwp
✅ 변환 성공: 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.hwp
✅ 변환 성공: 조선대학교_(재공고)2024 조선대학교 SW중심대학 사업관리시스템(WeHub) 구.hwp
✅ 변환 성공: 국민연금공단_2024년 이러닝시스템 운영 용역.hwp
✅ 변환 성공: 한국보건산업진흥원_의료기기산업 종합정보시스템(정보관리기관) 기능.hwp
✅ 변환 성공: 울산광역시_2024년 버스정보시스템 확대 구축 및 기능개선 용역.hwp
✅ 변환 성공: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).hwp
✅ 변환 성공: 부산관광공사_경영정보시스템 기능개선.hwp
✅ 변환 성공: 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.hwp
✅ 변환 성공: 고양도시관리공사_관산근린공원 다목적구장 홈페이지 및 회원 통합운영.hwp
✅ 변환 성공: 한국재정정보원_e나라도움 업무시스템 웹 접근성 컨설팅.hwp
✅ 변환 성공: 한국생산기술연구원_2세대 전자조달시스템  기반구축사업.hwp
✅ 변환 성공: 한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp
✅ 변환 성공: 한국지식재산보호원_IP-NAVI  해외지식재산센터 사업관리 시스템 기능개.hwp
✅ 변환 성공: 대한상공회의소_기업 재생에너지 지원센터 홈페이지 개편 및 시스템 고.hwp
✅ 변환 성공: 인천광역시 동구_수도국산달동네박물관 전시해설 시스템 구축(협상에 .hwp
✅ 변환 성공: 대한장애인체육회_2025년 전국장애인체육대회 전산 및 시스템, 홈페이지 .hwp
✅ 변환 성공: 인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp
✅ 변환 성공: 경희대학교_[

In [ ]:
# import os
# import json
# from glob import glob

# def merge_all_json_chunks(input_dir, output_file):
#     """
#     input_dir 내 모든 JSON 파일을 읽어 하나의 리스트로 합치고 output_file로 저장
#     """
#     all_chunks = []

#     json_files = glob(os.path.join(input_dir, "*.json"))
#     print(f"🔹 총 {len(json_files)}개 JSON 파일 발견")

#     for jf in json_files:
#         with open(jf, "r", encoding="utf-8") as f:
#             data = json.load(f)
#             all_chunks.extend(data)

#     print(f"🔹 총 {len(all_chunks)}개 청크 합쳐짐")

#     os.makedirs(os.path.dirname(output_file), exist_ok=True)
#     with open(output_file, "w", encoding="utf-8") as f:
#         json.dump(all_chunks, f, ensure_ascii=False, indent=2)

#     print(f"✅ 모든 청크 저장 완료: {output_file}")


# if __name__ == "__main__":
#     input_json_dir = "/content/drive/MyDrive/코드잇/projectmission2/data/processed/bydatapreprocessingbjs(hwp5proc)/json"
#     output_json_file = "/content/drive/MyDrive/코드잇/projectmission2/data/processed/bydatapreprocessingbjs(hwp5proc)/json/all_chunks.json"

#     merge_all_json_chunks(input_json_dir, output_json_file)

In [9]:
import os
import json
from tqdm import tqdm
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
# from langchain_chroma import Chroma

# ==============================
# 환경 변수 & API 키
# ==============================
load_dotenv("../notebooks/api_key.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

# ==============================
# JSON 청크 불러오기
# ==============================
json_folder = "../data/processed/bydatapreprocessingbjs(0922)/json"
all_chunks = []

json_files = [f for f in os.listdir(json_folder) if f.lower().endswith(".json")]
for jf in tqdm(json_files, desc="Loading JSON chunks"):
    with open(os.path.join(json_folder, jf), "r", encoding="utf-8") as f:
        chunks = json.load(f)
        file_prefix = os.path.splitext(jf)[0]
        for c in chunks:
            original_id = c.get("chunk_id", None)
            c["chunk_id"] = f"{file_prefix}_{original_id}"
        all_chunks.extend(chunks)

# ==============================
# 임베딩 생성 준비
# ==============================
embeddings = OpenAIEmbeddings(
    openai_api_key=openai_api_key,
    model="text-embedding-3-small"
)

# 메모리 내 Chroma 초기화
vectorstore = Chroma(embedding_function=embeddings, collection_name="hwp_chunks")

# ==============================
# 배치 단위 임베딩 처리
# ==============================
BATCH_SIZE = 100  # 필요에 따라 조절

for i in tqdm(range(0, len(all_chunks), BATCH_SIZE), desc="Vectorizing batches"):
    batch = all_chunks[i:i+BATCH_SIZE]
    texts = [c["content"] for c in batch]
    ids = [c.get("chunk_id", f"chunk_{i+j:06d}") for j, c in enumerate(batch)]
    vectorstore.add_texts(texts=texts, ids=ids)

print("✅ 메모리 내 Chroma DB 생성 완료")

# ==============================
# GPT-4.1-mini + RAG 연결
# ==============================
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

llm = ChatOpenAI(
    model_name="gpt-4.1-mini",
    temperature=0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

Vectorizing batches: 100%|██████████| 3403/3403 [1:07:39<00:00,  1.19s/it]

✅ 메모리 내 Chroma DB 생성 완료


In [ ]:
# from dotenv import load_dotenv
# import os
# import json
# from glob import glob
# from tqdm import tqdm

# # ❗ OpenAI, Chroma, LangChain import
# from langchain.vectorstores import Chroma
# from langchain_openai import OpenAIEmbeddings
# from langchain.chat_models import ChatOpenAI
# from langchain.chains import RetrievalQA

# # ==============================
# # 1️⃣ 환경 변수 & API 키
# # ==============================
# load_dotenv("../notebooks/api_key.env")

# openai_api_key = os.getenv("OPENAI_API_KEY")
# if not openai_api_key:
#     raise ValueError("❌ OPENAI_API_KEY가 없습니다!")

# # ==============================
# # 2️⃣ JSON 청크 불러오기
# # ==============================
# json_folder = "../data/processed/bydatapreprocessingbjs(0922)/json"

# all_chunks = []
# json_files = [f for f in os.listdir(json_folder) if f.lower().endswith(".json")]
# print(f"🔹 총 {len(json_files)}개 JSON 파일 발견")

# for jf in tqdm(json_files, desc="Loading JSON chunks"):
#     with open(os.path.join(json_folder, jf), "r", encoding="utf-8") as f:
#         chunks = json.load(f)
#         # 각 청크에 고유 ID 추가 (파일명_청크ID)
#         for c in chunks:
#             original_id = c.get("chunk_id", None)
#             file_prefix = os.path.splitext(jf)[0]
#             c["chunk_id"] = f"{file_prefix}_{original_id}"
#         all_chunks.extend(chunks)

# print(f"🔹 총 {len(all_chunks)}개 청크 합쳐짐")

# # ==============================
# # 3️⃣ 임베딩 생성 (DB 저장 및 배치 처리)
# # ==============================
# embeddings = OpenAIEmbeddings(
#     openai_api_key=openai_api_key,
#     model="text-embedding-3-small"  # 최신 임베딩 모델
# )

# persist_directory = "../data/processed/bydatapreprocessingbjs(0922)/chromaDB"
# os.makedirs(persist_directory, exist_ok=True)

# # 배치 크기 지정 (대용량 처리 안정화)
# BATCH_SIZE = 5000

# for i in tqdm(range(0, len(all_chunks), BATCH_SIZE), desc="Vectorizing batches"):
#     batch = all_chunks[i:i+BATCH_SIZE]
#     texts = [c["content"] for c in batch]
#     ids = [c.get("chunk_id", f"chunk_{i+j:06d}") for j, c in enumerate(batch)]
    
#     vectorstore = Chroma.from_texts(
#         texts=texts,
#         embedding=embeddings,
#         ids=ids,
#         persist_directory=persist_directory,
#         collection_name="hwp_chunks"  # 컬렉션 이름
#     )
#     vectorstore.persist()  # DB에 저장

# # ==============================
# # 4️⃣ GPT-4.1-mini + RAG 연결
# # ==============================
# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":5})

# llm = ChatOpenAI(
#     model_name="gpt-4.1-mini",
#     openai_api_key=openai_api_key,
#     temperature=0
# )

# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever,
#     return_source_documents=True
# )

In [10]:
# ==============================
# 5️⃣ 질문 예시
# ==============================
query = "'경기도'가 제목에 들어간 제안요청서를 알려줘."
result = qa_chain(query)
print("=== Answer ===")
print(result['result'])
print("\n=== Source Docs ===")
for doc in result['source_documents']:
    print("-", doc.metadata.get("source", "unknown"))

/tmp/ipykernel_109205/1881171153.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(query)


=== Answer ===
제공된 정보에는 '경기도'가 제목에 들어간 제안요청서에 대한 구체적인 내용이 없습니다. 추가 정보를 주시면 더 도와드릴 수 있습니다.

=== Source Docs ===
- unknown
- unknown
- unknown
- unknown
- unknown


In [ ]:
# import shutil

# # 삭제할 폴더 경로를 지정하세요.
# folder_path = '/content/drive/MyDrive/코드잇/projectmission2/data/processed/bydatapreprocessingbjs(hwp5proc)/vectorstore'
# try:
#     shutil.rmtree(folder_path)
#     print(f"'{folder_path}' 폴더와 모든 내용이 성공적으로 삭제되었습니다.")
# except OSError as e:
#     print(f"오류: {e.strerror} - {e.filename}")